In [4]:
from texter.utils.config_utils import keras_data_config as kdc
from texter.utils.config_utils import keras_model_config as kmc

from texter.utils.io_utils import load_config, save_config, load_model, save_model 
from texter.utils.io_utils import save_text_model, load_text_model

from texter.utils import text_utils as tu

In [6]:
kdc_config_params = dict(
    mappings_path="legal_files.csv", 
    column="Doc_Type", 
    root="../texter_nb/data/legal", 
    num_class=3, 
    MAXLEN=2000,
    NUM_WORDS=2000)

save_config("kdc_doc.json", kdc_config_params)

save the hyperparams to a json file

In [ ]:
kmc_config_w2v = dict(pretrained_embeddings_path="../../data/others/GoogleNews-vectors-negative300-SLIM.bin",
                      word_index=word_index,
                      num_class=3, 
                      model_type='kim', 
                      embedding_type='word2vec', 
                      EMBEDDING_DIM=300, 
                      activation='selu')

save_config("kmc_w2v_doc.json", kmc_config_w2v)

load the  hyperparams from the json file.

In [11]:
kdc_config = load_config("kdc_doc.json")
kmc_config = load_config("kmc_doc.json")
kmc_w2v_config = load_config("kmc_w2v_doc.json")

get the model

In [18]:
x_train, x_test, y_train, y_test, tokenizer, word_index = kdc(**kdc_config)
kim_w2v = kmc(**kmc_w2v_config, MAX_SEQUENCE_LENGTH=2000)

Found 62 unique tokens.
Shape of X train and X validation tensor: (8, 2000) (3, 2000)
Shape of label train and validation tensor: (8, 3) (3, 3)


In [21]:
kim_w2v.fit(x_train, y_train, batch_size=128, epochs=8)

Epoch 1/8
8/8 [==============================] - 0s 32ms/step - loss: 0.6224 - acc: 1.0000
Epoch 2/8
8/8 [==============================] - 0s 31ms/step - loss: 0.5399 - acc: 1.0000
Epoch 3/8
8/8 [==============================] - 0s 30ms/step - loss: 0.5061 - acc: 1.0000
Epoch 4/8
8/8 [==============================] - 0s 33ms/step - loss: 0.4753 - acc: 1.0000
Epoch 5/8
8/8 [==============================] - 0s 31ms/step - loss: 0.4083 - acc: 1.0000
Epoch 6/8
8/8 [==============================] - 0s 31ms/step - loss: 0.4094 - acc: 1.0000
Epoch 7/8
8/8 [==============================] - 0s 31ms/step - loss: 0.3871 - acc: 1.0000
Epoch 8/8
8/8 [==============================] - 0s 31ms/step - loss: 0.3677 - acc: 1.0000


In [23]:
#save model and tokenizer
save_model("kim_w2v", kim_w2v)
save_text_model("kim_w2v", tokenizer)

In [24]:
kim_w2v.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 2000)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 2000, 300)    17400       input_4[0][0]                    
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 2000, 300, 1) 0           embedding_8[0][0]                
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 1998, 1, 100) 90100       reshape_7[0][0]                  
__________________________________________________________________________________________________
conv2d_11 

In [25]:
#sample predictions

s = """Ayahuasca emerged again in the early 1960s with the counterculture movement.\n 
Beat writers like William Burroughs, Allen Ginsberg, and Jack Kerouac all described \n
their experiences with ayahuasca, most famously in Burroughs's book The Yage Letters. """

In [26]:
kim_w2v.predict(tu.keras_text_vectorizer(s, load_text_model("kim_w2v")))[0]

array([0.35382104, 0.3660751 , 0.28010386], dtype=float32)